# Optical character recognition
This project demonstrates how to predict character from a 5x5 matrix, represented as as vector of n=25 using an only mathematical approach. Only the charracters J, D and N will be used for recognition. Inner products, correlation matrices and dot products are at the center of this project.
By Jaden van Rijswijk.

In [30]:
import numpy as np
import random

In [60]:
def visualise_character(character):
    for i in range(5):
        for j in range(5):
            if character[i*5 + j] > 0.5:
                print("█", end="")
            else:
                print(" ", end="")
        print()
        
def visualise_probability_matrix(character):
    for i in range(5):
        for j in range(5):
            print(f"{character[i*5 + j]:.2f} ", end="")
        print()
        
def make_blurred(character, blur_level=0.2, min_value=0):
    blurred_character = []
    for value in character:
        if value > 0.5:
            blurred_character.append(random.uniform(max(min_value, value - blur_level), 1))
        else:
            blurred_character.append(value)
            
    assert len(blurred_character) == 25
    return blurred_character

def make_noisy(character, noise_level=0.2, max_value=1):
    noisy_character = []
    for value in character:
        if value < 0.5:
            noisy_character.append(random.uniform(0, min(value + noise_level, max_value)))
        else:
            noisy_character.append(value)
            
    assert len(noisy_character) == 25
    return noisy_character

In [61]:
j_perfect = [1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,1,1,1]
d_perfect = [1,1,1,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,1,1,1,0]
n_perfect = [1,0,0,0,1,1,1,0,0,1,1,0,1,0,1,1,0,0,1,1,1,0,0,0,1]

j_blurred = make_blurred(j_perfect, blur_level=0.3)
j_noisy = make_noisy(j_perfect, noise_level=0.3)
j_noisy_blurred = make_noisy(j_blurred, noise_level=0.3)

d_blurred = make_blurred(d_perfect, blur_level=0.3)
d_noisy = make_noisy(d_perfect, noise_level=0.3)
d_noisy_blurred = make_noisy(d_blurred, noise_level=0.3)

n_blurred = make_blurred(n_perfect, blur_level=0.3)
n_noisy = make_noisy(n_perfect, noise_level=0.3)
n_noisy_blurred = make_noisy(n_blurred, noise_level=0.3)

assert len(j_perfect) == 25
assert len(d_perfect) == 25
assert len(n_perfect) == 25

In [64]:
print(visualise_character(j_perfect))
print(visualise_probability_matrix(j_noisy_blurred))

█████
    █
    █
█   █
 ████
None
0.76 0.77 0.78 0.86 0.76 
0.09 0.19 0.29 0.29 0.84 
0.19 0.16 0.05 0.07 0.97 
0.99 0.15 0.11 0.28 0.71 
0.27 0.87 0.84 0.88 0.71 
None
